# `nanoGPT`: GPT-2 Small (150M Params)

## Install / Setup

### Google Colab

1. Mount Google Drive

    ```python
    from google.colab import drive
    drive.mount('/content/drive')
    ```

2. `bfloat16` doesn't work on Colab, switch to `float16`

3. To prepare data
    ```bash
    !python3 drive/MyDrive/Google-Colab/nanoGPT/data/prepare.py yelp_review_full
    ```

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

#cd drive/MyDrive/Google-Colab/
echo "pwd: $(pwd)"

HF_DATASETS_CACHE="./.cache/huggingface"
mkdir -p "${HF_DATASETS_CACHE}"

python3 -c 'import ngpt; print(ngpt.__file__)' 2> '/dev/null'
STATUS=$?

if [[ $STATUS -eq 0 ]]; then
    echo "Has ngpt installed. Nothing to do."
else
    echo "Does not have ngpt installed. Installing..."
    git clone 'https://github.com/saforem2/nanoGPT'
    python3 -m pip install -e nanoGPT
fi

#python3 nanoGPT/data/prepare.py yelp_review_full

#cd -

pwd: /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/notebooks
/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py
Has ngpt installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from enrich import get_logger
log = get_logger('jupyter')
log.info(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2023-11-30 07:10:16][INFO][3434626787.py:7] - /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
#import os
#os.listdir('../data/yelp_review_full/')

In [4]:
import os
import numpy as np
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config, PROJECT_ROOT
from ngpt.trainer import Trainer
from enrich.console import get_console

console = get_console()
HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['MASTER_PORT'] = '5278'
os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

SEED = np.random.randint(2**32)
console.log(f'SEED: {SEED}')

rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=yelp',
        #'data.dataset=yelp_review_full',
        #'data.root_path=../data/yelp_review_full/',
        #'data.out_dir=output_yelp_review_full',
        'model=gpt2_medium',
        'optimizer=gpt2_medium',
        'train=gpt2_medium',
        'train.dtype=bfloat16',
        'train.max_iters=1000',
        'train.log_interval=100',
        'train.init_from=gpt2-medium',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu23
  Local device: mlx5_0
--------------------------------------------------------------------------


[2023-11-30 07:10:33][INFO][configs.py:72] - Setting HF_DATASETS_CACHE to /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/.cache/huggingface/datasets
Failed to download font: Source Sans Pro, skipping!
Failed to download font: Titillium WebRoboto Condensed, skipping!


[07:10:36] SEED: 3164946669                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

[2023-11-30 07:10:37][INFO][configs.py:295] - Loading test from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/yelp_review_full/test.bin
[2023-11-30 07:10:37][INFO][configs.py:295] - Loading train from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/yelp_review_full/train.bin
[2023-11-30 07:10:37][WARNING][configs.py:330] - No meta.pkl found, assuming GPT-2 encodings...
[2023-11-30 07:10:38][INFO][configs.py:270] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-30 07:10:38][INFO][configs.py:432] - Tokens per iteration: 4,096
[2023-11-30 07:10:38][INFO][configs.py:454] - Using <torch.amp.autocast_mode.autocast object at 0x7f2cc6117fd0>
[2023-11-30 07:10:38][INFO][trainer.py:187] - Initializing from OpenAI GPT-2 Weights: gpt2-medium
[2023-11-30 07:10:45][INFO][model.py:225] - loading weights from pretrained gpt: gpt2-medium
[2023-11-30 07:10:45][INFO][model.py:234] - forcing vocab_size=50257, blo

## Prompt (**prior** to training)

In [5]:
query = "What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.."
outputs = trainer.evaluate(query, num_samples=1, display=False)
console.print(fr'\[prompt]: "{query}"')
console.print("\[response]:\n\n" + fr"{outputs['0']['raw']}")

[prompt]: "What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.."

[response]:

What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.. When I ask you to describe your computer, this is what you know: a computer that runs a quantum computer to run a computer program. There are now two supercomputers that can do quantum computations, so how can you explain quantum computing? We will get into what quantum computing is first, and then we are going to talk to you about what is a supercomputer. You will learn how to explain quantum computing to a child, and you will find out that quantum computing is what is called artificial intelligence (AI). Do you understand quantum computing?

Follow up – Answering the Quantum Computing Questions

Cognitive scientists are trying to define what is called artificial intelligence (AI). They are trying to understand what computers can do. At the heart of the argument are the following three questions:

What is AI and how does it apply to science?

What if I say, "A computer can tell me if I am an elf, a lion, a fox, or a squirrel?". Can I make a decision?

How does AI fit into the human condition?

Have you ever thought about what is intelligence? Have you ever wondered, "What if I want to create a computer that can build me an AI, and that computer is smarter than me? How will the AI affect my life?".

Before you even begin…

BECAUSE YOU HAVE NOT BEEN PURCHASED

This is where you may want to get an answer to a question that you have never had the opportunity to ask, and which is too big to ask right now. Think about it. Will I be able to make decisions? Will I be able to ask questions? Do I have the right to be an AI?

With an intelligent AI, you can have the ability to run my brain. You can have your mind, and you can control it. You can control your body, you can control your thoughts, and you can control your actions. That will make you a living being. If you are able to experience the world like the rest of us, you will have a better understanding of what it means to be alive; a living being. You will have the ability to observe, dream, and experience. All this will be possible to you if you are created as an intelligent being.

By creating an AI, this will be easier. It will be easy for someone to create an AI, and then run it as if

## Train Model


|  **NAME**  |     **DESCRIPTION**          |
|:----------:|:----------------------------:|
|   `step`   | Current training step        |
|   `loss`   | Loss value                   |
|   `dt`     | Time per step (in **ms**)    |
|   `sps`    | Samples per second           |
|   `mtps`   | (million) Tokens per sec     |
|   `mfu`    | Model Flops Utilization*     |
^Logging Legend

*in units of A100 `bfloat16` peak FLOPS

In [ ]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

## Evaluate Model

In [ ]:
query = "What is a supercomputer? Explain like I'm a child, and speak clearly. Double check your logic.."
outputs = trainer.evaluate(query, num_samples=1, display=False)
console.print(fr'\[prompt]: "{query}"')
console.print("\[response]:\n\n" + fr"{outputs['0']['raw']}")

## Train a bit more??

In [7]:
trainer.model.module.train()
for iter in range(10):
    console.rule(f'iter: {iter}')
    trainer.train(train_iters=100)
    query = "What is a supercomputer?"
    outputs = trainer.evaluate(query, num_samples=1, display=False)
    console.print(fr'\[prompt]: "{query}"')
    console.print("\[response]:\n\n" + fr"{outputs['0']['raw']}")
    console.rule()

───────────────────────────────────────────────────── iter: 0 ─────────────────────────────────────────────────────

  0%|          | 0/100 [00:00<?, ?it/s]

[2023-11-15 09:58:35][INFO][trainer.py:516] - step=1100 loss=2.939 dt=356.182 sps=2.808 mtps=0.011 mfu=26.810 train_loss=0.000 val_loss=0.000


[prompt]: "What is a supercomputer?"

[response]:

What is a supercomputer?

A supercomputer consists of two parts: “a processor and a network of interconnected graphics processors, called supercomputers,” said Richard Andrews, a computer scientist at the University of Edinburgh.

Image copyright PA Image caption The first three are the biggest computers in the world, but they’re all fitted with a touchscreen

The processor is a small computer, usually a single-core one. Usually it is made of silicon and it’s the first, and most expensive, part of the complex system. The other two parts of the system are: a chip to power the processor and a memory and software stack to allow for the system to interact on the internet.

The first is the processor. It runs a complex combination of algorithms. These algorithms work to compress higher-level data, but also do some processing to understand the lower levels of the system.

According to the Oxford computer science textbook, algorithms are:

It’s a special structure which allows a system to achieve a particular level of efficiency, such as a faster processor or a faster system of atoms or molecules.

Researchers use algorithms to implement certain types of systems like computers or medicine. But a supercomputer can also work faster, or faster, to solve problems.

Image copyright PA Image caption The supercomputer in the US can run things like social networks and video games

A supercomputer has a processor inside, but there is also a memory and software to process it. This allows a system to do something faster. This is called a supercomputer’s memory.

Image copyright Getty Images Image caption Computing power has increased, but it’s also improved the power of computers in recent years

The second piece of the system is the computer network. This is a computer system that works together to perform calculations.

This is a huge system. It is where all the computers in the world - computers in different countries, continents and different time zones - work together.

The network works and converts the data on the computers into calculations. This is the core of a supercomputer.

What goes on inside a supercomputer?

Image copyright PA Image caption The supercomputer works together to work out human intelligence

It can interact with the computer network via a touchscreen.

The other pieces of the system are:

A computer system can send and receive data through a touchscreen. It is

...

  0%|          | 0/100 [00:00<?, ?it/s]

[2023-11-15 10:06:30][INFO][trainer.py:516] - step=2000 loss=3.133 dt=378.236 sps=2.644 mtps=0.011 mfu=25.247 train_loss=0.000 val_loss=0.000


[prompt]: "What is a supercomputer?"

[response]:

What is a supercomputer?

Supercomputing is the concept of processing and storing information such as programs and images into a computer.

What kind of information can supercomputing access?

When used in supercomputing, it can access the entire computer.

What is the most powerful reason to allow supercomputing to access data?

Supercomputing can access all the data that a computer can read.

What are the most challenging aspects of supercomputing?

Most supercomputing tasks require only a basic understanding of CPU instructions.

What are the advantages of using supercomputing?

It’s a fully accessible method, which makes it accessible and easy to use.<|endoftext|>The Denver Broncos had a four-year contract with former Denver Broncos offensive coordinator Bill O’Brien out. The Broncos never hired Broncos offensive coordinator Bill O’Brien as Broncos offensive coordinator. O’Brien brought in Broncos offensive coordinator Mike Klis to be the Broncos offensive coordinator. Klis was a Broncos offensive coordinator for two seasons.

The Broncos hired Broncos offensive coordinator Mike Klis to be the Broncos offensive coordinator. Denver fired Bill O’Brien as Broncos offensive coordinator.

 Denver had a four-year contract with Bill O’Brien out. The Broncos never hired Bill O’Brien as Broncos offensive coordinator. O’Brien brought in Broncos offensive coordinator Mike Klis to be the Broncos offensive coordinator.

In 2009, the Broncos hired Mike Klis, who was the Broncos offensive coordinator for two seasons.

 Broncos coach Bill O’Brien, left, and Mike Klis, right, in the Broncos offense. In 2009, Broncos coach Bill O’Brien, left, and Mike Klis, right, in the Broncos offense.

It’s a matter of fact that Bill O’Brien brought in Peyton Manning to be the Broncos offensive coordinator. Broncos offensive coordinator Mike Klis , left, and Mike Klis, right, in the Broncos offense.

DENVER Broncos coach Bill O’Brien, left, and Mike Klis, right, were the Broncos offensive coordinators. Broncos coach Bill O’Brien, left, and Mike Klis, right.

Denver coaches Mike Klis and Mike Klis and Broncos offensive coordinator Mike Klis.

Denver coaches Mike Klis and Mike Klis, left. Denver coaches Mike Klis and Mike Klis

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────